In [1]:
from math import log2
import re 
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.cluster import KMeans
import numpy as np
import warnings
warnings.filterwarnings("ignore")
import shutil
import os
import math

##### define different functions of calculating differential score
def ks_score(r1, r2):
    r = r1-r2
    max_ks = max(max(r), 0)
    min_ks = min(min(r), 0)
    return (abs(max_ks)+abs(min_ks))/r1.sum()

def entropy_score(r_true:[],r_false:[]):

    h = 0.0
    for i in range(0,len(r_true)):
        if r_false[i]!=0:
            h = -r_true[i] * log2(r_false[i])+h
        else:
            h = -r_true[i] * log2(1/100000) + h
    return h


def KL(p,q):
    return sum(_p*math.log(_p/_q) for (_p,_q) in zip(p,q) if _p != 0 )

def JS(p,q):
    M = [0.5*(_p +_q) for (_p,_q) in zip(p,q)]
    return 0.5*(KL(p,M)+KL(q,M))


In [2]:
def read_d(dir_name,sim_type):######this function is uesd to collect the results of each bit.
    
    def read_file(name):
        aa=pd.read_csv(name,encoding='ANSI',error_bad_lines=False)
        getcol=[columns for columns in aa]
        del aa
        temp=[]
        for c in getcol:
            temp.append(remove(c))
        return dict(temp)

    def remove(text):
        punctuation = '[{} ？(!),;+"\'' 
        text = re.sub(r'[{}]+'.format(punctuation), '', text)
        text = re.sub(r'[a-z]+', '', text)
        text = re.sub(r'[A-Z]+', '', text)
        text=re.split(":",text)####the result is saved in csv file and there are punctuations in the front and end of the rusult.
        return text
    

    files=os.listdir(str(dir_name))
    whole_list=[]
    result=[]
    name=[]
    for file in files:
        if  sim_type.search(file):
            name.append(file)
#             print(file)
            result.append(read_file(dir_name+"/"+file))
    temp_len=[]
    for i in result:
        temp_len.append(len(i))

    full_dic=result[temp_len.index(max(temp_len))].keys()  ####some of the file lack some keys for no shots on it. Find the full key as a standard example

    for dic in result:
        if len(full_dic - dic.keys()) != 0:
            for key in full_dic - dic.keys():
                dic[key]=0 ###fulfill all lack keys
    for ele in result:
        tem=sorted(list(ele.items() ), key=lambda x:x[0]) #####set the results in ascending order of key 
        pp=[]
        for i in tem:
            pp.append(  float(i[1]) )
        whole_list.append(pp)#####save the name and result together
    return whole_list,name

In [3]:
####JS score with standardised answers
def formal_js(a,b):
    thershold=0.5
    candidates = [] # save right answer candidates
    answer = -1
    for results in a:
        flag = -1
        for i in range(0,len(candidates)):
            if JS(candidates[i]/(candidates[i].sum()),np.asarray(results)/(np.asarray(results).sum()))<thershold:
                flag = i
                candidates[i] = candidates[i] + np.asarray(results)/(np.asarray(results).sum())
                if candidates[i].sum() > candidates[answer].sum():
                    answer = i
                break

        if flag == -1:
            candidates.append(np.asarray(results)/(np.asarray(results).sum()))
            if answer == -1:
                answer = 0

    wrong_out = []           
    print("Right answer:",candidates[answer]/candidates[answer].sum()*1024)

    result=[]

    for i in range(0, len(a)):
        k = JS(np.asarray(a[i])/np.asarray(a[i]).sum(),candidates[answer]/candidates[answer].sum())
        result.append([b[i],k])
        if k > thershold:
            print(k)
            print(b[i]+":"+str(a[i])+":"+str(k))
            wrong_out.append([b[i],k])
            
    return result,wrong_out

noise_file = re.compile("startCirq_noise_[0-9]+"+".csv") 
js_error=[]
for i in range(1,100):
    print("reading folder"+str(i))
    print("------------------------------------------")
    a,b=read_d(str(i),noise_file)
    js_error.append(["folder"+str(i),formal_js(a,b)])
for i in range(0,len(js_error)):
    if len(js_error[i][1][1])>0:
        print(js_error[i][0])

reading folder1
------------------------------------------
Right answer: [ 13.13391304  13.34857143  78.27875776  76.74434783  50.02335404
  50.81043478 117.14782609 113.42708075  13.16571429  13.16571429
  78.70807453  77.03850932  49.06931677  49.49863354 115.13639752
 115.30335404]
reading folder2
------------------------------------------
Right answer: [ 13.13391304  13.3247205   60.0089441   60.39850932  67.95925466
  66.56       116.12223602 114.38111801  13.29291925  12.67279503
  60.80397516  61.3684472   67.77639752  66.83031056 115.02509317
 114.34136646]
reading folder3
------------------------------------------
Right answer: [120.24049689 121.18658385 121.03552795 121.31378882   7.96621118
   7.17913043   7.26658385   6.88496894 118.72993789 120.05763975
 121.2978882  121.13888199   7.85490683   7.44944099   7.46534161
   6.93267081]
reading folder4
------------------------------------------
Right answer: [ 12.24347826  13.11006211  51.70086957  51.51006211  76.09242236
  7

Right answer: [2.54409938e-01 2.06708075e-01 9.22236025e-01 9.38136646e-01
 3.45043478e+00 3.64919255e+00 1.44218634e+01 1.50658385e+01
 5.10409938e+00 5.45391304e+00 2.20621118e+01 2.43279503e+01
 8.53306832e+01 8.75249689e+01 3.79945342e+02 3.75342112e+02]
reading folder31
------------------------------------------
Right answer: [ 17.64173913  16.87850932  10.62956522  10.53416149   7.92645963
   8.53068323   1.81267081   1.96372671 215.50906832 213.27503106
 142.31055901 140.94310559  94.58484472  94.46559006  23.2626087
  23.73167702]
reading folder32
------------------------------------------
Right answer: [ 31.98409938  31.8489441    2.89391304   3.11652174   2.11478261
   2.2578882    1.39130435   1.61391304 395.47229814 393.37341615
  36.22956522  36.87354037  24.12124224  23.09565217  17.7689441
  19.84397516]
reading folder33
------------------------------------------
Right answer: [ 13.84944099 218.71304348  10.60571429 150.45962733   8.8884472
  84.04273292   6.2489441   19

Right answer: [  2.95751553   3.21192547   6.16944099   6.04223602  18.96149068
  19.78037267  22.03031056  22.04621118  27.87378882  27.98509317
  47.98807453  49.08521739 181.01267081 182.10186335 203.15428571
 203.59950311]
reading folder60
------------------------------------------
Right answer: [217.95776398  16.87055901 178.15850932  12.29118012  58.30757764
   6.38409938  18.64347826   2.79850932 218.25192547  16.39354037
 178.07900621  13.04645963  58.85614907   6.40795031  18.65937888
   2.89391304]
reading folder61
------------------------------------------
Right answer: [ 12.82385093  13.29291925  47.08173913  47.34409938  81.03751553
  79.70981366 115.70086957 115.5736646   12.35478261  13.42012422
  46.9068323   46.81937888  80.60024845  79.83701863 116.1063354
 115.39080745]
reading folder62
------------------------------------------
Right answer: [ 10.62956522  10.58186335  33.11304348  32.28621118  94.25093168
  95.89664596 117.06832298 116.27329193  10.78062112  10.534

b'Skipping line 6: expected 16 fields, saw 20\n'
b'Skipping line 6: expected 16 fields, saw 20\n'


Right answer: [114.39701863 115.82807453  25.51254658  25.91006211  13.07031056
  13.26111801 102.11378882 101.73217391 113.41913043 115.96322981
  24.94012422  25.2978882   13.69043478  12.58534161 104.45913043
 101.81962733]
reading folder83
------------------------------------------
Right answer: [ 11.83801242  11.58360248  17.18857143  15.59055901 112.10732919
 111.82906832 115.08869565 117.4578882   12.32298137  11.61540373
  16.23453416  15.5826087  111.61440994 110.81937888 114.95354037
 118.17341615]
reading folder84
------------------------------------------
Right answer: [  8.10136646   7.28248447  64.14310559  64.65192547  63.86484472
  63.22086957 121.68745342 119.83503106   7.79925466   8.10136646
  63.86484472  63.61043478  63.34807453  63.34807453 120.46310559
 120.67776398]
reading folder85
------------------------------------------
Right answer: [ 22.92869565  24.2484472   85.72819876  87.7157764  150.76968944
 150.86509317 213.82360248 214.76968944   1.85242236   1.83

In [4]:
##JS score with trimmed mean 
def js_ternn(a,b):
    thershold=0.6
    candidates = [] 
    for i in range(0,len(a[0])):
        temp=[]
        for c in range(0,len(a)):
            temp.append(a[c][i])        
        fla=sorted(temp)[4:-4]
        candidates.append( sum(fla)/len(fla)  )
    print("Right answer:", np.asarray(candidates)/ np.asarray(candidates).sum()*1024)
    max_diff_name = ''
    result=[]
    wrong_out = [] 
    for i in range(0, len(a)):
        k = JS(np.asarray(a[i])/np.asarray(a[i]).sum(), np.asarray(candidates)/ np.asarray(candidates).sum())
        result.append([b[i],k])
        if k > thershold:
            print(k)
            print(b[i]+":"+str(a[i])+":"+str(k))
            wrong_out.append([b[i],k])
    return result,wrong_out
noise_file = re.compile("startCirq_noise_[0-9]+"+".csv") 

js_ter=[]
for i in range(1,100):
    print("reading folder"+str(i))
    print("------------------------------------------")
    a,b=read_d(str(i),noise_file)
    js_ter.append(["folder"+str(i),js_ternn(a,b)])
for i in range(0,len(js_ter)):
    if len(js_ter[i][1][1])>0:
        print(js_ter[i][0])

reading folder1
------------------------------------------
Right answer: [ 13.29440086  13.5274208   86.16832146  84.18151985  41.37943589
  41.95585364 117.58922091 114.16750704  13.36798611  13.33119348
  86.31549195  84.79473022  40.75396132  41.25679382 116.10525181
 115.81091083]
reading folder2
------------------------------------------
Right answer: [ 13.2047558   13.18023444  58.593805    58.56928364  68.94182162
  67.54410374 117.45734504 114.710952    13.58483698  12.60398233
  59.15779643  60.04056562  68.51269771  67.47053964 115.39755026
 115.02972976]
reading folder3
------------------------------------------
Right answer: [119.82228136 120.85628074 121.33070398 121.71997434   7.99220699
   7.11634869   7.17717218   6.84872532 118.81261137 120.50350448
 121.24555109 120.91710423   7.83406591   7.37180736   7.45696025
   6.9947017 ]
reading folder4
------------------------------------------
Right answer: [ 12.44682889  12.91281855  46.0226097   45.55662004  80.97183368
  8

Right answer: [1.83007470e-01 2.19608965e-01 9.27237850e-01 8.90636356e-01
 3.46494144e+00 3.58694642e+00 1.43111842e+01 1.52872240e+01
 5.20961266e+00 5.42922162e+00 2.16802850e+01 2.44253971e+01
 8.49520678e+01 8.72701624e+01 3.80496932e+02 3.75665535e+02]
reading folder31
------------------------------------------
Right answer: [ 17.74448391  16.88726246  11.1561248   11.05815664   6.75980339
   7.11493799   1.80016503   1.91037922 216.11777228 214.92990827
 152.78135352 151.287339    83.67705904  83.66481302  23.2551932
  23.85524821]
reading folder32
------------------------------------------
Right answer: [ 31.88483996  32.14109011   2.84315641   3.18482328   2.06220358
   2.22083462   1.26904836   1.56190567 396.18713506 393.86868133
  36.38752115  37.31490264  24.66102624  23.08691818  16.50983103
  18.81608237]
reading folder33
------------------------------------------
Right answer: [ 13.77326379 219.41896908  11.01127833 165.58469489   8.38372579
  67.90084617   6.4038955   

Right answer: [  2.80989764   3.06645351   3.0908874    2.99315183  20.58555442
  21.25748646  21.58734401  21.53847622  27.6102985   27.80576964
  27.26822401  29.16185068 202.37371448 203.63205994 205.2324799
 203.98635138]
reading folder60
------------------------------------------
Right answer: [219.02943851  16.81259873 207.14298023  13.68780815  30.01024365
   3.8722895   18.07476904   2.7939304  219.8014456   16.15087837
 206.62830884  14.47206931  30.45139055   3.98257623  18.29534249
   2.7939304 ]
reading folder61
------------------------------------------
Right answer: [ 13.18407519  13.71192922  38.5456202   38.8402364   88.76540753
  87.10819138 116.44705516 115.95602815  12.11609144  13.14724816
  38.75430668  38.38603642  88.27438052  87.77107784 117.19587135
 115.79644437]
reading folder62
------------------------------------------
Right answer: [ 10.55202623  10.2578931   17.59896594  17.20678842 109.09888216
 110.9494698  117.43265433 117.22431003  10.52751514  10.368

b'Skipping line 6: expected 16 fields, saw 20\n'
b'Skipping line 6: expected 16 fields, saw 20\n'


Right answer: [114.9457765  116.01167743  12.83981814  13.40339794  12.43551089
  12.71730079 114.17391721 113.84312036 114.92127303 117.01631969
  13.19511845  13.15836324  13.10935631  12.2272314  117.12658531
 112.87523331]
reading folder83
------------------------------------------
Right answer: [ 11.83532916  11.50589217  12.78703604  11.12764969 116.6817039
 116.09603813 115.19313673 117.71882038  12.14036342  11.59130176
  12.1891689   10.90802502 116.08383676 115.54697647 114.9979148
 117.59680667]
reading folder84
------------------------------------------
Right answer: [  8.06521531   7.20590774  65.49151242  65.25827179  62.17704038
  61.71055913 122.78277549 120.68360986   7.70921646   7.97928455
  64.86544548  64.26393018  61.3668361   61.84559317 121.34650427
 121.24829769]
reading folder85
------------------------------------------
Right answer: [ 22.76643557  24.17403777  70.75955056  72.47315324 165.81553909
 165.46057853 215.30193641 215.97513746   1.81152283   1.8360

In [5]:
#JS score with 2-means

def js_2mean(a,b):
    def two_means(x):
        y =  np.asarray(x).reshape(-1,1)
        km = KMeans(2)
        km.fit(y)
        km.cluster_centers_
        return float( (km.cluster_centers_[1]+km.cluster_centers_[0])/2 )

    thershold=0.5
    k_mean_candidates=[]
    for i in range(0,len(a[0])):
        temp=[]
        for c in range(0,len(a)):
            temp.append(a[c][i])        
        k_mean_candidates.append( temp )

    k_means_answer=[]
    for i in k_mean_candidates:
        k_means_answer.append(two_means(np.asarray(i)))

    print("Right answer:",np.asarray(k_means_answer)/ np.asarray(k_means_answer).sum()*1024)
    wrong_out=[]
    result=[]
    for i in range(0, len(a)):
        k = JS(np.asarray(a[i])/np.asarray(a[i]).sum(), np.asarray(k_means_answer)/ np.asarray(k_means_answer).sum())
        result.append([b[i],k])
        if k > thershold:
            print(k)
            print(b[i]+":"+str(a[i])+":"+str(k))
            wrong_out.append([b[i],k])
    return result,wrong_out

noise_file = re.compile("startCirq_noise_[0-9]+"+".csv") 
js_2means=[]
for i in range(1,100):
    print("reading folder"+str(i))
    print("------------------------------------------")
    a,b=read_d(str(i),noise_file)
    js_2means.append(["folder"+str(i),js_2mean(a,b)])
for i in range(0,len(js_2means)):
    if len(js_2means[i][1][1])>0:
        print(js_2means[i][0])

reading folder1
------------------------------------------
Right answer: [ 13.51174785  13.1065077   62.28906363  61.39372426  65.49862766
  66.40762475 118.8444144  113.61777145  12.86136915  12.77900037
  62.69651892  61.57051585  64.15941241  64.63591506 114.24857242
 116.37921414]
reading folder2
------------------------------------------
Right answer: [ 12.89278846  13.52872412  62.10119816  62.45513386  65.57903117
  64.29324914 114.56698084 116.0544745   12.83497511  13.00009243
  62.90515757  63.45679954  65.42211046  64.53035841 115.87260262
 114.50632362]
reading folder3
------------------------------------------
Right answer: [121.17964081 121.20859989 118.20929008 120.08610745   7.95647092
   8.04815328   7.33806712   7.16280468 119.66599892 118.50792565
 122.39662866 122.67809683   8.11235544   7.55162163   7.24806749
   6.65017115]
reading folder4
------------------------------------------
Right answer: [ 11.61755159  13.58410752  63.10348851  62.35418591  65.15444091
  6

Right answer: [8.28752974e-01 2.50697774e-01 8.32620487e-01 1.14400410e+00
 3.41584351e+00 3.90632714e+00 1.44306612e+01 1.45184942e+01
 5.05677439e+00 5.89072739e+00 2.28104587e+01 2.39416375e+01
 8.68334866e+01 8.79764692e+01 3.78579883e+02 3.73583162e+02]
reading folder31
------------------------------------------
Right answer: [ 17.43891557  16.9053206    9.07585973   9.03751001   9.65834136
  10.38843325   1.91169747   2.06799068 216.44156312 212.94377244
 121.29365918 120.27124111 114.68954758 114.19751341  23.73992522
  23.93870925]
reading folder32
------------------------------------------
Right answer: [ 31.27458242  30.86144179   2.85977043   2.97054826   2.07000825
   2.20509915   1.65419549   1.74029053 398.23196892 393.87310517
  35.00210157  36.21900526  22.61674255  21.8549258   19.11927913
  21.4469353 ]
reading folder33
------------------------------------------
Right answer: [ 13.79887486 219.70527174   9.62972615 120.72645261  10.23415974
 113.64038451   5.80884235 

Right answer: [  3.55935083   3.617195    15.07866794  15.23870637  12.82288825
  13.52705731  22.75670188  24.41127458  28.00118184  28.58554808
 107.9497278  107.1297214  118.69516488 118.95884723 198.64578926
 205.02217735]
reading folder60
------------------------------------------
Right answer: [219.06315954  17.41006304 121.61779166   9.30580148 111.62207208
  11.22887231  20.21732575   2.81204622 215.28858928  17.91258037
 121.88283351   9.49699579 112.28669993  11.17829181  19.46084965
   3.21602758]
reading folder61
------------------------------------------
Right answer: [ 12.14971564  12.82167381  62.48273432  62.5555757   65.07012107
  64.32729068 115.91174678 117.68537673  12.95438533  15.00796132
  62.10486962  61.92124863  64.81062363  64.43503523 114.19025479
 115.57138673]
reading folder62
------------------------------------------
Right answer: [ 10.91932374  12.41719396  62.3850819   60.84809036  64.96896576
  66.14621438 117.50785352 114.68610191  11.92951467  11.41

b'Skipping line 6: expected 16 fields, saw 20\n'
b'Skipping line 6: expected 16 fields, saw 20\n'


Right answer: [114.34619152 116.30679417  62.12854899  62.20154121  15.34458778
  14.61473794  66.60844681  66.38947013 110.45939503 113.32311311
  59.03398277  60.58354689  15.48043442  13.69011545  67.59232118
  65.89677261]
reading folder83
------------------------------------------
Right answer: [ 11.84811957  11.59542965  63.58672643  62.05815952  65.19814249
  65.31413131 115.33088944 116.50649669  13.38911385  11.70582083
  60.21890828  63.0109248   64.76732688  64.61405595 115.51650278
 119.33925155]
reading folder84
------------------------------------------
Right answer: [  8.17501074   7.15020342  61.45602597  61.91532723  66.05386607
  65.38905163 120.9838142  120.32093076   7.67708955   8.34349547
  61.17465222  60.95948406  65.50766997  65.51663531 120.70716943
 122.66957397]
reading folder85
------------------------------------------
Right answer: [ 23.442535    25.35700386 113.92817234 116.21540915 121.31577193
 122.07716799 212.64214236 212.1669143    2.12148981   1.74

In [6]:
####KS score with standardised answers
def formal_ks(a,b):
    thershold=0.6
    candidates = [] # save right answer candidates
    answer = -1
    for results in a:
        flag = -1
        for i in range(0,len(candidates)):
            if ks_score(candidates[i]/(candidates[i].sum()),np.asarray(results)/(np.asarray(results).sum()))<thershold:
                flag = i
                candidates[i] = candidates[i] + np.asarray(results)/(np.asarray(results).sum())
                if candidates[i].sum() > candidates[answer].sum():
                    answer = i
                break

        if flag == -1:
            candidates.append(np.asarray(results)/(np.asarray(results).sum()))
            if answer == -1:
                answer = 0

    wrong_out = []           
    print("Right answer:",candidates[answer]/candidates[answer].sum()*1024)

    result=[]

    for i in range(0, len(a)):
        k = ks_score(np.asarray(a[i])/np.asarray(a[i]).sum(),candidates[answer]/candidates[answer].sum())
        result.append([b[i],k])
        if k > thershold:
            print(k)
            print(b[i]+":"+str(a[i])+":"+str(k))
            wrong_out.append([b[i],k])
            
    return result,wrong_out

noise_file = re.compile("startCirq_noise_[0-9]+"+".csv") 
ks_error=[]
for i in range(1,100):
    print("reading folder"+str(i))
    print("------------------------------------------")
    a,b=read_d(str(i),noise_file)
    ks_error.append(["folder"+str(i),formal_ks(a,b)])
for i in range(0,len(ks_error)):
    if len(ks_error[i][1][1])>0:
        print(ks_error[i][0])

reading folder1
------------------------------------------
Right answer: [ 13.13391304  13.34857143  78.27875776  76.74434783  50.02335404
  50.81043478 117.14782609 113.42708075  13.16571429  13.16571429
  78.70807453  77.03850932  49.06931677  49.49863354 115.13639752
 115.30335404]
reading folder2
------------------------------------------
Right answer: [ 13.13391304  13.3247205   60.0089441   60.39850932  67.95925466
  66.56       116.12223602 114.38111801  13.29291925  12.67279503
  60.80397516  61.3684472   67.77639752  66.83031056 115.02509317
 114.34136646]
reading folder3
------------------------------------------
Right answer: [120.24049689 121.18658385 121.03552795 121.31378882   7.96621118
   7.17913043   7.26658385   6.88496894 118.72993789 120.05763975
 121.2978882  121.13888199   7.85490683   7.44944099   7.46534161
   6.93267081]
reading folder4
------------------------------------------
Right answer: [ 12.24347826  13.11006211  51.70086957  51.51006211  76.09242236
  7

Right answer: [2.54409938e-01 2.06708075e-01 9.22236025e-01 9.38136646e-01
 3.45043478e+00 3.64919255e+00 1.44218634e+01 1.50658385e+01
 5.10409938e+00 5.45391304e+00 2.20621118e+01 2.43279503e+01
 8.53306832e+01 8.75249689e+01 3.79945342e+02 3.75342112e+02]
reading folder31
------------------------------------------
Right answer: [ 17.64173913  16.87850932  10.62956522  10.53416149   7.92645963
   8.53068323   1.81267081   1.96372671 215.50906832 213.27503106
 142.31055901 140.94310559  94.58484472  94.46559006  23.2626087
  23.73167702]
reading folder32
------------------------------------------
Right answer: [ 31.98409938  31.8489441    2.89391304   3.11652174   2.11478261
   2.2578882    1.39130435   1.61391304 395.47229814 393.37341615
  36.22956522  36.87354037  24.12124224  23.09565217  17.7689441
  19.84397516]
reading folder33
------------------------------------------
Right answer: [ 13.84944099 218.71304348  10.60571429 150.45962733   8.8884472
  84.04273292   6.2489441   19

Right answer: [ 14.3026087   13.38037267   0.71552795   0.74732919 232.02186335
 234.12074534   9.22236025   9.23826087  14.17540373  13.45192547
   0.63602484   0.58037267 229.43801242 233.09515528   9.7152795
   9.15875776]
reading folder49
------------------------------------------
Right answer: [  1.76496894   1.75701863   7.0042236    6.82136646  11.93341615
  11.49614907  16.82285714  16.43329193  20.4084472   20.11428571
  76.85565217  75.0668323  161.72521739 160.5247205  217.13888199
 218.13267081]
reading folder50
------------------------------------------
Right answer: [117.3068323  118.04621118 117.50559006 117.09217391  11.15428571
  10.53416149  10.36720497  11.38484472 117.29093168 117.82360248
 116.48       116.52770186  11.03503106  10.47055901  10.55801242
  10.42285714]
reading folder51
------------------------------------------
Right answer: [ 27.89180952   1.7432381  407.94209524  24.73447619  23.06438095
  10.72761905  11.264        1.34095238  29.20838095   2.121

Right answer: [231.58459627 233.21440994   9.06335404   9.01565217  14.63652174
  12.26732919   0.51677019   0.44521739 231.05987578 233.74708075
   9.50857143   9.39726708  15.32024845  13.09416149   0.54062112
   0.58832298]
reading folder67
------------------------------------------
Right answer: [  5.80372671   5.97863354   5.50161491   5.22335404   5.31080745
   5.47776398   5.44596273   5.75602484 123.60347826 123.72273292
 121.95776398 121.65565217 121.31378882 121.50459627 123.07875776
 122.66534161]
reading folder68
------------------------------------------
Right answer: [  1.93192547   2.44074534   8.77714286   8.33192547  11.70285714
  11.25763975  18.55602484  17.93590062  22.5868323   24.28819876
  85.91900621  84.13018634 152.36770186 149.3863354  213.57714286
 210.81043478]
reading folder69
------------------------------------------
Right answer: [116.38459627 119.15130435  95.93639752  94.83925466  31.82509317
  32.20670807  10.79652174  11.32919255 116.79801242 116.90

b'Skipping line 6: expected 16 fields, saw 20\n'
b'Skipping line 6: expected 16 fields, saw 20\n'


Right answer: [114.39701863 115.82807453  25.51254658  25.91006211  13.07031056
  13.26111801 102.11378882 101.73217391 113.41913043 115.96322981
  24.94012422  25.2978882   13.69043478  12.58534161 104.45913043
 101.81962733]
reading folder83
------------------------------------------
Right answer: [ 11.83801242  11.58360248  17.18857143  15.59055901 112.10732919
 111.82906832 115.08869565 117.4578882   12.32298137  11.61540373
  16.23453416  15.5826087  111.61440994 110.81937888 114.95354037
 118.17341615]
reading folder84
------------------------------------------
Right answer: [  8.10136646   7.28248447  64.14310559  64.65192547  63.86484472
  63.22086957 121.68745342 119.83503106   7.79925466   8.10136646
  63.86484472  63.61043478  63.34807453  63.34807453 120.46310559
 120.67776398]
reading folder85
------------------------------------------
Right answer: [ 22.92869565  24.2484472   85.72819876  87.7157764  150.76968944
 150.86509317 213.82360248 214.76968944   1.85242236   1.83

In [7]:
##KS score with trimmed mean
def ks_ternn(a,b):
    thershold=0.6
    candidates = [] # save right answer candidates

    for i in range(0,len(a[0])):
        temp=[]
        for c in range(0,len(a)):
            temp.append(a[c][i])        
        fla=sorted(temp)[4:-4]
        candidates.append( sum(fla)/len(fla)  )
    print("Right answer:", np.asarray(candidates)/ np.asarray(candidates).sum()*1024)
    max_diff_name = ''
    result=[]
    wrong_out = [] 
    for i in range(0, len(a)):
        k = ks_score(np.asarray(a[i])/np.asarray(a[i]).sum(), np.asarray(candidates)/ np.asarray(candidates).sum())
        result.append([b[i],k])
        if k > thershold:
            print(k)
            print(b[i]+":"+str(a[i])+":"+str(k))
            wrong_out.append([b[i],k])
    return result,wrong_out

noise_file = re.compile("startCirq_noise_[0-9]+"+".csv") 
ks_ter=[]
for i in range(1,100):
    print("reading folder"+str(i))
    print("------------------------------------------")
    a,b=read_d(str(i),noise_file)
    ks_ter.append(["folder"+str(i),ks_ternn(a,b)])
for i in range(0,len(ks_ter)):
    if len(ks_ter[i][1][1])>0:
        print(ks_ter[i][0])

reading folder1
------------------------------------------
Right answer: [ 13.29440086  13.5274208   86.16832146  84.18151985  41.37943589
  41.95585364 117.58922091 114.16750704  13.36798611  13.33119348
  86.31549195  84.79473022  40.75396132  41.25679382 116.10525181
 115.81091083]
reading folder2
------------------------------------------
Right answer: [ 13.2047558   13.18023444  58.593805    58.56928364  68.94182162
  67.54410374 117.45734504 114.710952    13.58483698  12.60398233
  59.15779643  60.04056562  68.51269771  67.47053964 115.39755026
 115.02972976]
reading folder3
------------------------------------------
Right answer: [119.82228136 120.85628074 121.33070398 121.71997434   7.99220699
   7.11634869   7.17717218   6.84872532 118.81261137 120.50350448
 121.24555109 120.91710423   7.83406591   7.37180736   7.45696025
   6.9947017 ]
reading folder4
------------------------------------------
Right answer: [ 12.44682889  12.91281855  46.0226097   45.55662004  80.97183368
  8

Right answer: [1.83007470e-01 2.19608965e-01 9.27237850e-01 8.90636356e-01
 3.46494144e+00 3.58694642e+00 1.43111842e+01 1.52872240e+01
 5.20961266e+00 5.42922162e+00 2.16802850e+01 2.44253971e+01
 8.49520678e+01 8.72701624e+01 3.80496932e+02 3.75665535e+02]
reading folder31
------------------------------------------
Right answer: [ 17.74448391  16.88726246  11.1561248   11.05815664   6.75980339
   7.11493799   1.80016503   1.91037922 216.11777228 214.92990827
 152.78135352 151.287339    83.67705904  83.66481302  23.2551932
  23.85524821]
reading folder32
------------------------------------------
Right answer: [ 31.88483996  32.14109011   2.84315641   3.18482328   2.06220358
   2.22083462   1.26904836   1.56190567 396.18713506 393.86868133
  36.38752115  37.31490264  24.66102624  23.08691818  16.50983103
  18.81608237]
reading folder33
------------------------------------------
Right answer: [ 13.77326379 219.41896908  11.01127833 165.58469489   8.38372579
  67.90084617   6.4038955   

Right answer: [  2.80989764   3.06645351   3.0908874    2.99315183  20.58555442
  21.25748646  21.58734401  21.53847622  27.6102985   27.80576964
  27.26822401  29.16185068 202.37371448 203.63205994 205.2324799
 203.98635138]
reading folder60
------------------------------------------
Right answer: [219.02943851  16.81259873 207.14298023  13.68780815  30.01024365
   3.8722895   18.07476904   2.7939304  219.8014456   16.15087837
 206.62830884  14.47206931  30.45139055   3.98257623  18.29534249
   2.7939304 ]
reading folder61
------------------------------------------
Right answer: [ 13.18407519  13.71192922  38.5456202   38.8402364   88.76540753
  87.10819138 116.44705516 115.95602815  12.11609144  13.14724816
  38.75430668  38.38603642  88.27438052  87.77107784 117.19587135
 115.79644437]
reading folder62
------------------------------------------
Right answer: [ 10.55202623  10.2578931   17.59896594  17.20678842 109.09888216
 110.9494698  117.43265433 117.22431003  10.52751514  10.368

b'Skipping line 6: expected 16 fields, saw 20\n'
b'Skipping line 6: expected 16 fields, saw 20\n'


Right answer: [114.9457765  116.01167743  12.83981814  13.40339794  12.43551089
  12.71730079 114.17391721 113.84312036 114.92127303 117.01631969
  13.19511845  13.15836324  13.10935631  12.2272314  117.12658531
 112.87523331]
reading folder83
------------------------------------------
Right answer: [ 11.83532916  11.50589217  12.78703604  11.12764969 116.6817039
 116.09603813 115.19313673 117.71882038  12.14036342  11.59130176
  12.1891689   10.90802502 116.08383676 115.54697647 114.9979148
 117.59680667]
reading folder84
------------------------------------------
Right answer: [  8.06521531   7.20590774  65.49151242  65.25827179  62.17704038
  61.71055913 122.78277549 120.68360986   7.70921646   7.97928455
  64.86544548  64.26393018  61.3668361   61.84559317 121.34650427
 121.24829769]
reading folder85
------------------------------------------
Right answer: [ 22.76643557  24.17403777  70.75955056  72.47315324 165.81553909
 165.46057853 215.30193641 215.97513746   1.81152283   1.8360

Right answer: [117.77164426 120.55163545 119.45427051 118.46664206   9.01058548
   9.21786552   9.33979496   8.54725361 118.27155496 118.6739221
 119.21041163 119.58839289   9.44953145   8.71795482   8.86427015
   8.86427015]
reading folder97
------------------------------------------
Right answer: [229.95819488 233.92193995   9.75691095   9.65934184  14.45242434
  12.48884601   0.62200307   0.52443396 232.60475697 232.98283727
   9.39102679   9.73251867  14.64756256  12.14735413   0.60980693
   0.50004169]
reading folder98
------------------------------------------
Right answer: [  2.65570322   3.07180419   4.49144278   4.23443924  19.88717851
  19.99732288  22.33483125  22.91002964  25.54125633  26.98537145
  37.33894254  36.26197533 191.99388087 193.51142557 204.88077254
 207.90362367]
reading folder99
------------------------------------------
Right answer: [ 12.15066805  38.35054605  22.9539838  243.05010706  16.65817395
 188.32310618  27.70646284 394.4557601    0.86965467   3.294

In [8]:
##KS score with 2-means

def ks_2mean(a,b):
    def two_means(x):
        y =  np.asarray(x).reshape(-1,1)
        km = KMeans(2)
        km.fit(y)
        km.cluster_centers_
        return float( (km.cluster_centers_[1]+km.cluster_centers_[0])/2 )

    thershold=0.6
    k_mean_candidates=[]
    for i in range(0,len(a[0])):
        temp=[]
        for c in range(0,len(a)):
            temp.append(a[c][i])        
        k_mean_candidates.append( temp )

    k_means_answer=[]
    for i in k_mean_candidates:
        k_means_answer.append(two_means(np.asarray(i)))

    print("Right answer:",np.asarray(k_means_answer)/ np.asarray(k_means_answer).sum()*1024)
    wrong_out=[]
    result=[]
    for i in range(0, len(a)):
        k = ks_score(np.asarray(a[i])/np.asarray(a[i]).sum(), np.asarray(k_means_answer)/ np.asarray(k_means_answer).sum())
        result.append([b[i],k])
        if k > thershold:
            print(k)
            print(b[i]+":"+str(a[i])+":"+str(k))
            wrong_out.append([b[i],k])
    return result,wrong_out

noise_file = re.compile("startCirq_noise_[0-9]+"+".csv") 
ks_2means=[]
for i in range(1,100):
    print("reading folder"+str(i))
    print("------------------------------------------")
    a,b=read_d(str(i),noise_file)
    ks_2means.append(["folder"+str(i),ks_2mean(a,b)])
for i in range(0,len(ks_ter)):
    if len(ks_2means[i][1][1])>0:
        print(ks_2means[i][0])

reading folder1
------------------------------------------
Right answer: [ 12.82940355  13.11535802  62.33112507  61.43518111  65.54285639
  66.45246729 118.92466551 113.69449321  12.87005394  12.78762954
  62.7388555   61.61209208  64.20273682  64.67956123 114.32572013
 116.4578006 ]
reading folder2
------------------------------------------
Right answer: [ 12.89278846  13.52872412  62.10119816  62.45513386  65.57903117
  64.29324914 114.56698084 116.0544745   12.83497511  13.00009243
  62.90515757  63.45679954  65.42211046  64.53035841 115.87260262
 114.50632362]
reading folder3
------------------------------------------
Right answer: [121.17964081 121.20859989 118.20929008 120.08610745   7.95647092
   8.04815328   7.33806712   7.16280468 119.66599892 118.50792565
 122.39662866 122.67809683   8.11235544   7.55162163   7.24806749
   6.65017115]
reading folder4
------------------------------------------
Right answer: [ 11.61755159  13.58410752  63.10348851  62.35418591  65.15444091
  6

Right answer: [8.28541336e-01 2.50633754e-01 8.32407863e-01 1.14371196e+00
 3.41497121e+00 3.90532959e+00 1.44269760e+01 1.45147866e+01
 5.05548305e+00 6.15072022e+00 2.28046336e+01 2.39355235e+01
 8.68113120e+01 8.79540028e+01 3.78483206e+02 3.73487760e+02]
reading folder31
------------------------------------------
Right answer: [ 17.43891557  16.9053206    9.07585973   9.03751001   9.65834136
  10.38843325   1.91169747   2.06799068 216.44156312 212.94377244
 121.29365918 120.27124111 114.68954758 114.19751341  23.73992522
  23.93870925]
reading folder32
------------------------------------------
Right answer: [ 31.27031746  30.85723317   2.9990246    2.97014317   2.06972596
   2.20479843   1.6539699    1.7400532  398.17766153 393.8193922
  34.99732829  36.21406603  22.61365828  21.85194542  19.11667181
  21.44401055]
reading folder33
------------------------------------------
Right answer: [ 13.79887486 219.70527174   9.62972615 120.72645261  10.23415974
 113.64038451   5.80884235  

Right answer: [  3.56002644   3.42351234  15.08153008  15.24159888  12.82532221
  13.52962493  22.76102142  24.41590817  28.00649685  28.59097401
 107.97021814 107.15005609 118.71769485 118.98142726 198.68349496
 205.06109337]
reading folder60
------------------------------------------
Right answer: [219.14008377  17.05659807 121.66049786   9.30906922 111.66126827
  11.23281534  20.22442508   2.81303367 215.36418807  17.91887039
 121.92563278   9.50033067 112.3261295   11.18221708  19.46768335
   3.21715689]
reading folder61
------------------------------------------
Right answer: [ 12.15186852  12.64249666  62.49380603  62.56666033  65.08165126
  64.33868924 115.93228592 117.70623015  12.9566808   15.01062068
  62.11587438  61.93222085  64.82210784  64.44645288 114.21048889
 115.59186556]
reading folder62
------------------------------------------
Right answer: [ 10.91562214  12.41298459  62.36393362  60.82746312  64.94694156
  66.1237911  117.46801886 114.64722382  11.92547061  11.40

b'Skipping line 6: expected 16 fields, saw 20\n'
b'Skipping line 6: expected 16 fields, saw 20\n'


Right answer: [114.34619152 116.30679417  62.12854899  62.20154121  15.34458778
  14.61473794  66.60844681  66.38947013 110.45939503 113.32311311
  59.03398277  60.58354689  15.48043442  13.69011545  67.59232118
  65.89677261]
reading folder83
------------------------------------------
Right answer: [ 11.84704236  11.59437541  63.58094524  62.0525173   65.19221479
  65.30819306 115.58394532 116.49590413  13.38789654  11.53431523
  60.21343328  63.00519596  64.76143836  64.60818135 115.50600023
 119.32840144]
reading folder84
------------------------------------------
Right answer: [  8.17501074   7.15020342  61.45602597  61.91532723  66.05386607
  65.38905163 120.9838142  120.32093076   7.67708955   8.34349547
  61.17465222  60.95948406  65.50766997  65.51663531 120.70716943
 122.66957397]
reading folder85
------------------------------------------
Right answer: [ 23.442535    25.35700386 113.92817234 116.21540915 121.31577193
 122.07716799 212.64214236 212.1669143    2.12148981   1.74

Right answer: [  3.06812447   3.35690711  14.36273409  13.40929302  12.78690787
  13.27992655  23.86299062  22.58529021  26.72221602  26.12718846
 109.57951203 111.00152456 118.16964937 120.22932656 204.07861627
 201.37979279]
reading folder99
------------------------------------------
Right answer: [ 11.65145767  37.31001441  21.30725631 207.84467686  17.97569125
 225.63902976  26.8865735  395.21569153   1.13050501   3.37730433
   1.91881279  19.35259757   1.50611575  16.64517381   2.33868906
  33.90041039]
folder91


In [9]:
####CS score with standardised answers
def cross_entropy(a,b):
    thershold=4
    candidates = [] # save right answer candidates
    answer = 1
    for results in a:
        candidates.append(np.asarray(results))
    wrong_out = []      
    result=[] 
    print("Right answer:",candidates[answer]/candidates[answer].sum()*1024)
    for i in range(0, len(a)):
        k = entropy_score(candidates[answer]/candidates[answer].sum(),np.asarray(a[i])/np.asarray(a[i]).sum())-\
        entropy_score(candidates[answer]/candidates[answer].sum(),candidates[answer]/candidates[answer].sum())
        result.append([b[i],k])
        if k > thershold:
            print(k)
            print(b[i]+":"+str(a[i])+":"+str(k))
            wrong_out.append([b[i],k])
    return result,wrong_out

noise_file = re.compile("startCirq_noise_[0-9]+"+".csv") 
cross_error=[]
for i in range(1,100):
    print("reading folder"+str(i))
    print("------------------------------------------")
    a,b=read_d(str(i),noise_file)
    cross_error.append(["folder"+str(i),cross_entropy(a,b)])
for i in range(0,len(ks_error)):
    if len(cross_error[i][1][1])>0:
        print(cross_error[i][0])

reading folder1
------------------------------------------
Right answer: [ 12.43428571  13.89714286 120.86857143 101.85142857  14.44571429
  14.99428571 114.10285714 115.74857143  14.08        13.16571429
 113.92       117.94285714  13.34857143  15.36       112.45714286
 115.38285714]
reading folder2
------------------------------------------
Right answer: [ 15.72571429  14.99428571 109.16571429 106.60571429  15.90857143
  11.15428571 108.25142857 121.78285714  14.26285714  13.89714286
 118.30857143 113.55428571  13.53142857  17.55428571 115.56571429
 113.73714286]
reading folder3
------------------------------------------
Right answer: [125.07428571 114.46857143 120.13714286 122.14857143   8.04571429
   6.58285714   7.86285714   5.66857143 121.41714286 121.96571429
 120.86857143 121.6          9.50857143   6.58285714   6.4
   5.66857143]
reading folder4
------------------------------------------
Right answer: [ 10.05714286  11.70285714  12.06857143  12.98285714 111.54285714
 118.85714

Right answer: [1.82857143e-01 1.82857143e-01 5.48571429e-01 1.28000000e+00
 1.28000000e+00 3.47428571e+00 1.31657143e+01 1.68228571e+01
 3.84000000e+00 5.85142857e+00 1.86514286e+01 2.61485714e+01
 8.52114286e+01 9.12457143e+01 3.75588571e+02 3.80525714e+02]
reading folder31
------------------------------------------
Right answer: [ 14.08        16.09142857   2.56         3.29142857  18.28571429
  22.30857143   1.64571429   2.37714286 219.42857143 210.28571429
  22.49142857  25.23428571 207.72571429 207.72571429  23.58857143
  26.88      ]
reading folder32
------------------------------------------
Right answer: [ 31.45142857  29.44         1.28         2.56         1.09714286
   0.54857143   2.56         3.10857143 409.23428571 398.26285714
  28.52571429  27.24571429  11.33714286  10.42285714  35.29142857
  31.63428571]
reading folder33
------------------------------------------
Right answer: [ 12.06857143 216.86857143  12.06857143 217.78285714   5.85142857
  18.28571429   4.57142857 

Right answer: [102.58285714  97.82857143  25.96571429  22.12571429 108.06857143
 103.31428571  26.88        26.51428571 105.50857143 107.88571429
  25.05142857  21.21142857  97.46285714 102.4         24.68571429
  26.51428571]
reading folder56
------------------------------------------
Right answer: [119.77142857 112.45714286  14.81142857  13.89714286 115.56571429
 112.09142857  13.71428571  12.43428571 114.83428571 109.53142857
  13.53142857  13.34857143 118.85714286 115.56571429  11.70285714
  11.88571429]
reading folder57
------------------------------------------
Right answer: [111.72571429 119.58857143 118.67428571 115.93142857  11.33714286
  10.78857143   8.22857143   9.69142857 121.96571429 119.77142857
 118.85714286 121.78285714   9.87428571   8.41142857  10.05714286
   7.31428571]
reading folder58
------------------------------------------
Right answer: [  9.32571429  44.61714286  25.96571429 390.03428571  12.98285714
  39.31428571  28.34285714 389.30285714   1.28         4.57

Right answer: [  9.69142857   7.68        10.24         8.22857143 119.58857143
 118.67428571 115.74857143 123.24571429   7.31428571   9.69142857
   9.87428571   8.22857143 117.57714286 117.39428571 124.34285714
 116.48      ]
reading folder77
------------------------------------------
Right answer: [ 31.08571429  26.33142857   5.66857143   6.4         21.94285714
  24.50285714   5.12         5.30285714 193.09714286 193.28
  33.09714286  36.20571429 185.41714286 186.69714286  36.02285714
  33.82857143]
reading folder78
------------------------------------------
Right answer: [124.16       124.16         7.31428571   6.21714286   6.03428571
   7.31428571 117.76       118.30857143 115.74857143 119.77142857
   6.21714286   9.32571429   7.68         7.86285714 122.69714286
 123.42857143]
reading folder79
------------------------------------------
Right answer: [ 13.71428571  17.00571429 104.22857143 119.58857143  16.64
  13.89714286 110.08       121.6         15.17714286  12.61714286
 112.

b'Skipping line 6: expected 16 fields, saw 20\n'
b'Skipping line 6: expected 16 fields, saw 20\n'


Right answer: [115.56571429 123.61142857   9.69142857  14.08         9.14285714
  11.15428571 115.01714286 114.10285714 114.83428571 115.38285714
  10.24        11.70285714  11.70285714  10.60571429 112.09142857
 125.07428571]
reading folder83
------------------------------------------
Right answer: [ 12.98285714  11.33714286  11.70285714   8.96       114.65142857
 116.11428571 114.83428571 123.61142857  15.54285714  10.42285714
  10.24         9.32571429 118.85714286 111.90857143 115.38285714
 118.12571429]
reading folder84
------------------------------------------
Right answer: [  6.58285714   4.93714286   5.30285714   5.12       117.21142857
 123.06285714 130.74285714 120.13714286   6.76571429   6.94857143
   6.21714286   5.85142857 121.41714286 115.93142857 113.92
 133.85142857]
reading folder85
------------------------------------------
Right answer: [ 27.97714286  23.77142857 207.36       202.05714286  22.85714286
  25.78285714 222.35428571 206.26285714   2.01142857   3.29142857

In [10]:
##CS score with trimmed mean
def cs_ter(a,b):
    thershold=4
    candidates = [] # save right answer candidates

    for i in range(0,len(a[0])):
        temp=[]
        for c in range(0,len(a)):
            temp.append(a[c][i])        
        fla=sorted(temp)[4:-4]
        candidates.append( sum(fla)/len(fla)  )
    print("Right answer:", np.asarray(candidates)/ np.asarray(candidates).sum()*1024)
    max_diff_name = ''
    result=[]
    wrong_out = [] 
    for i in range(0, len(a)):
        k = entropy_score(np.asarray(a[i])/np.asarray(a[i]).sum(), np.asarray(candidates)/ np.asarray(candidates).sum())-\
        entropy_score(np.asarray(candidates)/ np.asarray(candidates).sum(), np.asarray(candidates)/ np.asarray(candidates).sum())
        k= abs(k)
        result.append([b[i],k])
        if k > thershold:
            print(k)
            print(b[i]+":"+str(a[i])+":"+str(k))
            wrong_out.append([b[i],k])
    return result,wrong_out


noise_file = re.compile("startCirq_noise_[0-9]+"+".csv") 
cs_ternn=[]
for i in range(1,100):
    print("reading folder"+str(i))
    print("------------------------------------------")
    a,b=read_d(str(i),noise_file)
    cs_ternn.append(["folder"+str(i),cs_ter(a,b)])
for i in range(0,len(ks_error)):
    if len(cs_ternn[i][1][1])>0:
        print(cs_ternn[i][0])

reading folder1
------------------------------------------
Right answer: [ 13.29440086  13.5274208   86.16832146  84.18151985  41.37943589
  41.95585364 117.58922091 114.16750704  13.36798611  13.33119348
  86.31549195  84.79473022  40.75396132  41.25679382 116.10525181
 115.81091083]
reading folder2
------------------------------------------
Right answer: [ 13.2047558   13.18023444  58.593805    58.56928364  68.94182162
  67.54410374 117.45734504 114.710952    13.58483698  12.60398233
  59.15779643  60.04056562  68.51269771  67.47053964 115.39755026
 115.02972976]
reading folder3
------------------------------------------
Right answer: [119.82228136 120.85628074 121.33070398 121.71997434   7.99220699
   7.11634869   7.17717218   6.84872532 118.81261137 120.50350448
 121.24555109 120.91710423   7.83406591   7.37180736   7.45696025
   6.9947017 ]
reading folder4
------------------------------------------
Right answer: [ 12.44682889  12.91281855  46.0226097   45.55662004  80.97183368
  8

Right answer: [1.83007470e-01 2.19608965e-01 9.27237850e-01 8.90636356e-01
 3.46494144e+00 3.58694642e+00 1.43111842e+01 1.52872240e+01
 5.20961266e+00 5.42922162e+00 2.16802850e+01 2.44253971e+01
 8.49520678e+01 8.72701624e+01 3.80496932e+02 3.75665535e+02]
reading folder31
------------------------------------------
Right answer: [ 17.74448391  16.88726246  11.1561248   11.05815664   6.75980339
   7.11493799   1.80016503   1.91037922 216.11777228 214.92990827
 152.78135352 151.287339    83.67705904  83.66481302  23.2551932
  23.85524821]
reading folder32
------------------------------------------
Right answer: [ 31.88483996  32.14109011   2.84315641   3.18482328   2.06220358
   2.22083462   1.26904836   1.56190567 396.18713506 393.86868133
  36.38752115  37.31490264  24.66102624  23.08691818  16.50983103
  18.81608237]
reading folder33
------------------------------------------
Right answer: [ 13.77326379 219.41896908  11.01127833 165.58469489   8.38372579
  67.90084617   6.4038955   

Right answer: [  2.80989764   3.06645351   3.0908874    2.99315183  20.58555442
  21.25748646  21.58734401  21.53847622  27.6102985   27.80576964
  27.26822401  29.16185068 202.37371448 203.63205994 205.2324799
 203.98635138]
reading folder60
------------------------------------------
Right answer: [219.02943851  16.81259873 207.14298023  13.68780815  30.01024365
   3.8722895   18.07476904   2.7939304  219.8014456   16.15087837
 206.62830884  14.47206931  30.45139055   3.98257623  18.29534249
   2.7939304 ]
reading folder61
------------------------------------------
Right answer: [ 13.18407519  13.71192922  38.5456202   38.8402364   88.76540753
  87.10819138 116.44705516 115.95602815  12.11609144  13.14724816
  38.75430668  38.38603642  88.27438052  87.77107784 117.19587135
 115.79644437]
reading folder62
------------------------------------------
Right answer: [ 10.55202623  10.2578931   17.59896594  17.20678842 109.09888216
 110.9494698  117.43265433 117.22431003  10.52751514  10.368

b'Skipping line 6: expected 16 fields, saw 20\n'
b'Skipping line 6: expected 16 fields, saw 20\n'


Right answer: [114.9457765  116.01167743  12.83981814  13.40339794  12.43551089
  12.71730079 114.17391721 113.84312036 114.92127303 117.01631969
  13.19511845  13.15836324  13.10935631  12.2272314  117.12658531
 112.87523331]
reading folder83
------------------------------------------
Right answer: [ 11.83532916  11.50589217  12.78703604  11.12764969 116.6817039
 116.09603813 115.19313673 117.71882038  12.14036342  11.59130176
  12.1891689   10.90802502 116.08383676 115.54697647 114.9979148
 117.59680667]
reading folder84
------------------------------------------
Right answer: [  8.06521531   7.20590774  65.49151242  65.25827179  62.17704038
  61.71055913 122.78277549 120.68360986   7.70921646   7.97928455
  64.86544548  64.26393018  61.3668361   61.84559317 121.34650427
 121.24829769]
reading folder85
------------------------------------------
Right answer: [ 22.76643557  24.17403777  70.75955056  72.47315324 165.81553909
 165.46057853 215.30193641 215.97513746   1.81152283   1.8360

In [11]:
#CS score with  2-means

def cs_2mean(a,b):
    def two_means(x):
        y =  np.asarray(x).reshape(-1,1)
        km = KMeans(2)
        km.fit(y)
        km.cluster_centers_
        return float( (km.cluster_centers_[1]+km.cluster_centers_[0])/2 )

    thershold=4
    k_mean_candidates=[]
    for i in range(0,len(a[0])):
        temp=[]
        for c in range(0,len(a)):
            temp.append(a[c][i])        
        k_mean_candidates.append( temp )

    k_means_answer=[]
    for i in k_mean_candidates:
        k_means_answer.append(two_means(np.asarray(i)))

    print("Right answer:",np.asarray(k_means_answer)/ np.asarray(k_means_answer).sum()*1024)
    wrong_out=[]
    result=[]
    for i in range(0, len(a)):
        k = entropy_score(np.asarray(a[i])/np.asarray(a[i]).sum(), np.asarray(k_means_answer)/ np.asarray(k_means_answer).sum())-\
        entropy_score(np.asarray(k_means_answer)/ np.asarray(k_means_answer).sum(), np.asarray(k_means_answer)/ np.asarray(k_means_answer).sum())
        k= abs(k)
        result.append([b[i],k])
        if k > thershold:
            print(k)
            print(b[i]+":"+str(a[i])+":"+str(k))
            wrong_out.append([b[i],k])
    return result,wrong_out

noise_file = re.compile("startCirq_noise_[0-9]+"+".csv") 
cs_2ms=[]
for i in range(1,100):
    print("reading folder"+str(i))
    print("------------------------------------------")
    a,b=read_d(str(i),noise_file)
    cs_2ms.append(["folder"+str(i),cs_2mean(a,b)])
for i in range(0,len(ks_error)):
    if len(cs_2ms[i][1][1])>0:
        print(cs_2ms[i][0])

reading folder1
------------------------------------------
Right answer: [ 12.82940355  13.11535802  62.33112507  61.43518111  65.54285639
  66.45246729 118.92466551 113.69449321  12.87005394  12.78762954
  62.7388555   61.61209208  64.20273682  64.67956123 114.32572013
 116.4578006 ]
reading folder2
------------------------------------------
Right answer: [ 12.89278846  13.52872412  62.10119816  62.45513386  65.57903117
  64.29324914 114.56698084 116.0544745   12.83497511  13.00009243
  62.90515757  63.45679954  65.42211046  64.53035841 115.87260262
 114.50632362]
reading folder3
------------------------------------------
Right answer: [121.46557612 121.49460353 118.48821655 120.36946246   7.97524499
   8.06714369   7.35538201   7.17970602 119.94836265 118.78755679
 120.26920662 122.9675679    8.13149735   7.56944043   7.26517002
   6.66586288]
reading folder4
------------------------------------------
Right answer: [ 11.6110732   13.57653251  63.06829963  62.31941487  65.11810835
  6

Right answer: [8.28741720e-01 2.50694370e-01 8.32609181e-01 1.14398857e+00
 3.41579712e+00 3.90627410e+00 1.44304652e+01 1.45182970e+01
 5.05670573e+00 6.15220778e+00 2.25624933e+01 2.39413124e+01
 8.68323075e+01 8.79752746e+01 3.78574743e+02 3.73578089e+02]
reading folder31
------------------------------------------
Right answer: [ 17.1115297   16.91081909   9.07881167   9.04044948   9.66148276
  10.3918121    1.91231926   2.0686633  216.51196114 213.0130328
 121.33311017 120.31035956 114.72685057 114.23465636  23.74764667
  23.94649536]
reading folder32
------------------------------------------
Right answer: [ 31.27458242  30.86144179   2.85977043   2.97054826   2.07000825
   2.20509915   1.65419549   1.74029053 398.23196892 393.87310517
  35.00210157  36.21900526  22.61674255  21.8549258   19.11927913
  21.4469353 ]
reading folder33
------------------------------------------
Right answer: [ 13.79887486 219.70527174   9.62972615 120.72645261  10.23415974
 113.64038451   5.80884235  

Right answer: [  3.55935083   3.617195    15.07866794  15.23870637  12.82288825
  13.52705731  22.75670188  24.41127458  28.00118184  28.58554808
 107.9497278  107.1297214  118.69516488 118.95884723 198.64578926
 205.02217735]
reading folder60
------------------------------------------
Right answer: [219.28755904  17.06807669 121.74237204   9.31533397 111.73641324
  11.24037472  20.23803557   2.81492677 214.81999833  17.93092929
 122.00768539   9.50672413 112.40172191  11.1897424   19.48078457
   3.21932195]
reading folder61
------------------------------------------
Right answer: [ 12.15186852  12.64249666  62.49380603  62.56666033  65.08165126
  64.33868924 115.93228592 117.70623015  12.9566808   15.01062068
  62.11587438  61.93222085  64.82210784  64.44645288 114.21048889
 115.59186556]
reading folder62
------------------------------------------
Right answer: [ 10.9118902   12.40874071  62.34261202  60.80666682  64.92473686
  66.10118403 117.42785772 114.60802708  11.92139341  11.40

b'Skipping line 6: expected 16 fields, saw 20\n'
b'Skipping line 6: expected 16 fields, saw 20\n'


Right answer: [114.34619152 116.30679417  62.12854899  62.20154121  15.34458778
  14.61473794  66.60844681  66.38947013 110.45939503 113.32311311
  59.03398277  60.58354689  15.48043442  13.69011545  67.59232118
  65.89677261]
reading folder83
------------------------------------------
Right answer: [ 11.84369036  11.5910949   63.56295564  62.03496016  65.17376931
  65.28971476 115.84097254 116.46294278  13.38410856  11.53105171
  60.1963965   62.98736927  64.74311476  64.58990112 115.47331896
 119.29463867]
reading folder84
------------------------------------------
Right answer: [  8.17501074   7.15020342  61.45602597  61.91532723  66.05386607
  65.38905163 120.9838142  120.32093076   7.67708955   8.34349547
  61.17465222  60.95948406  65.50766997  65.51663531 120.70716943
 122.66957397]
reading folder85
------------------------------------------
Right answer: [ 23.43897893  25.35315737 113.91089019 116.19778004 121.29736913
 122.05864969 212.60988596 212.13472998   2.276502     1.74

In [12]:
 ####IQR method
def box_line(a,b):
    def box_line(x):
        a=sorted(x)
        if (len(a)+1)%4==0:

            q1 = a[ int((len(a)+1)*0.25)-1]
            q3 = a[ int((len(a)+1)*0.75)-1]
        else:
            q1 = (a[ int((len(a)+1)*0.25)-1] + a[ math.ceil((len(a)+1)*0.25)-1])/2
            q3 = (a[ int((len(a)+1)*0.75)-1] + a[ math.ceil((len(a)+1)*0.75)-1])/2
            print(q3,q1)
        iqr=q3-q1
        return ((q1-1.5*iqr),(q3+1.5*iqr))

    box_candidates=[]
    for i in range(0,len(a[0])):
        temp=[]
        for c in range(0,len(a)):
            temp.append(a[c][i])        
        box_candidates.append( temp )

    box_answer=[]
    for i in box_candidates:
        box_answer.append(box_line(i))

    print("Right answer:",box_answer)
    wrong_out=[]
    result=[]
    for i in range(0,len(a)):
        flag=0
        for c in range(0,len(a[0])):
            if a[i][c]<box_answer[c][0] or  a[i][c]>box_answer[c][1]:
                flag += 1
        result.append([b[i],flag])
        if flag >= 10:
            print(b[i]+":"+str(a[i])+":"+str(flag))
            wrong_out.append([b[i],flag])
    return result,wrong_out
noise_file = re.compile("startCirq_noise_[0-9]+"+".csv") 
box_l=[]
for i in range(1,100):
    print("reading folder"+str(i))
    print("------------------------------------------")
    a,b=read_d(str(i),noise_file)
    box_l.append(["folder"+str(i),box_line(a,b)])
for i in range(0,len(ks_ter)):
    if len(box_l[i][1][1])>0:
        print(box_l[i][0])

reading folder1
------------------------------------------
Right answer: [(32.0, 112.0), (42.5, 102.5), (-793.5, 1490.5), (-755.0, 1445.0), (-764.5, 1471.5), (-748.0, 1452.0), (591.5, 683.5), (556.5, 680.5), (30.0, 110.0), (44.0, 100.0), (-778.5, 1473.5), (-746.0, 1430.0), (-748.0, 1436.0), (-728.0, 1424.0), (572.0, 684.0), (575.0, 695.0)]
reading folder2
------------------------------------------
Right answer: [(41.5, 101.5), (34.5, 110.5), (-772.0, 1452.0), (-762.0, 1446.0), (-775.0, 1497.0), (-713.5, 1426.5), (563.5, 711.5), (558.5, 690.5), (35.5, 111.5), (34.5, 102.5), (-782.5, 1469.5), (-768.0, 1464.0), (-773.5, 1486.5), (-751.5, 1460.5), (567.0, 695.0), (560.0, 688.0)]
reading folder3
------------------------------------------
Right answer: [(588.5, 728.5), (587.0, 739.0), (620.5, 712.5), (587.0, 739.0), (22.5, 66.5), (27.0, 51.0), (17.5, 61.5), (14.0, 62.0), (593.0, 713.0), (570.5, 750.5), (566.0, 766.0), (598.0, 734.0), (27.0, 59.0), (11.5, 71.5), (21.0, 61.0), (24.5, 52.5)]
re

Right answer: [(557.5, 753.5), (593.5, 717.5), (596.0, 732.0), (553.5, 765.5), (11.5, 63.5), (28.0, 52.0), (18.5, 54.5), (25.0, 57.0), (573.5, 753.5), (595.5, 719.5), (609.5, 709.5), (611.0, 715.0), (12.5, 64.5), (25.0, 57.0), (20.0, 52.0), (24.5, 60.5)]
reading folder25
------------------------------------------
Right answer: [(-2.5, 17.5), (0.5, 20.5), (-121.5, 218.5), (-110.5, 205.5), (-91.5, 200.5), (-91.0, 205.0), (58.5, 118.5), (78.5, 106.5), (55.0, 191.0), (90.0, 170.0), (-1351.5, 2540.5), (-1373.5, 2590.5), (-1442.0, 2822.0), (-1360.0, 2712.0), (1105.0, 1241.0), (1075.5, 1263.5)]
reading folder26
------------------------------------------
Right answer: [(4.5, 32.5), (4.5, 32.5), (-181.5, 342.5), (-173.0, 339.0), (-140.5, 295.5), (-138.5, 297.5), (74.5, 182.5), (68.0, 196.0), (88.5, 204.5), (79.5, 227.5), (-1245.5, 2422.5), (-1215.0, 2393.0), (-1268.0, 2572.0), (-1313.0, 2647.0), (931.0, 1251.0), (997.0, 1221.0)]
reading folder27
------------------------------------------
Right 

Right answer: [(-2.0, 22.0), (-3.5, 24.5), (-135.0, 241.0), (-133.5, 238.5), (-99.5, 208.5), (-89.5, 194.5), (50.5, 134.5), (40.5, 140.5), (36.0, 196.0), (47.0, 175.0), (-1440.0, 2672.0), (-1411.5, 2608.5), (-1452.0, 2812.0), (-1475.5, 2840.5), (1035.0, 1339.0), (1025.5, 1357.5)]
reading folder50
------------------------------------------
Right answer: [(574.0, 710.0), (599.0, 687.0), (587.5, 695.5), (598.0, 678.0), (35.5, 87.5), (33.0, 81.0), (31.5, 83.5), (40.5, 84.5), (596.0, 684.0), (567.5, 723.5), (575.0, 695.0), (576.0, 696.0), (34.0, 90.0), (38.5, 74.5), (25.0, 89.0), (31.5, 83.5)]
reading folder51
------------------------------------------
Right answer: [(101.5, 217.5), (-1.0, 23.0), (-3223.0, 5521.0), (-185.0, 335.0), (-2829.0, 5051.0), (-104.5, 323.5), (-109.5, 342.5), (-73.5, 138.5), (106.5, 238.5), (0.0, 24.0), (-3206.5, 5525.5), (-183.5, 332.5), (-2876.5, 5119.5), (-71.5, 268.5), (-145.5, 402.5), (-79.0, 145.0)]
reading folder52
------------------------------------------
R

Right answer: [(558.0, 694.0), (546.5, 694.5), (46.0, 110.0), (44.0, 100.0), (574.0, 654.0), (544.5, 676.5), (57.0, 105.0), (40.5, 100.5), (573.0, 669.0), (534.5, 722.5), (49.0, 113.0), (46.5, 98.5), (560.0, 696.0), (538.5, 726.5), (62.5, 98.5), (51.5, 95.5)]
reading folder75
------------------------------------------
Right answer: [(-701.5, 1438.5), (-682.5, 1409.5), (-665.5, 1378.5), (-649.0, 1359.0), (-689.5, 1362.5), (-717.0, 1403.0), (-773.5, 1422.5), (-761.5, 1418.5), (-694.0, 1434.0), (-725.0, 1475.0), (-704.0, 1432.0), (-675.5, 1400.5), (-712.0, 1400.0), (-686.0, 1378.0), (-775.5, 1412.5), (-771.0, 1429.0)]
reading folder76
------------------------------------------
Right answer: [(20.5, 64.5), (24.5, 60.5), (-880.5, 1595.5), (-850.5, 1545.5), (609.0, 705.0), (576.0, 736.0), (-878.0, 1570.0), (-880.0, 1584.0), (24.0, 64.0), (25.0, 65.0), (-855.0, 1553.0), (-880.5, 1595.5), (575.0, 743.0), (594.0, 722.0), (-897.5, 1602.5), (-874.0, 1550.0)]
reading folder77
---------------------

b'Skipping line 6: expected 16 fields, saw 20\n'
b'Skipping line 6: expected 16 fields, saw 20\n'


Right answer: [(540.5, 712.5), (496.5, 772.5), (-3.5, 152.5), (-20.5, 183.5), (21.0, 117.0), (12.5, 136.5), (543.0, 703.0), (496.0, 736.0), (479.0, 751.0), (530.0, 730.0), (-5.5, 166.5), (-6.0, 162.0), (3.5, 151.5), (26.0, 114.0), (558.5, 706.5), (488.5, 740.5)]
reading folder83
------------------------------------------
Right answer: [(43.5, 87.5), (40.5, 84.5), (43.5, 95.5), (31.5, 91.5), (560.0, 712.0), (565.5, 705.5), (581.5, 681.5), (550.0, 734.0), (55.0, 79.0), (45.5, 81.5), (28.5, 104.5), (36.5, 88.5), (562.5, 702.5), (582.0, 678.0), (588.5, 672.5), (568.0, 720.0)]
reading folder84
------------------------------------------
Right answer: [(-17.5, 106.5), (-9.0, 87.0), (-889.5, 1570.5), (-917.0, 1603.0), (-883.0, 1605.0), (-879.5, 1604.5), (582.0, 750.0), (596.0, 716.0), (-5.0, 91.0), (0.0, 88.0), (-891.5, 1568.5), (-890.5, 1569.5), (-887.5, 1612.5), (-896.0, 1624.0), (600.5, 724.5), (603.5, 711.5)]
reading folder85
------------------------------------------
Right answer: [(84.0,